In [ ]:
from herbie import Herbie
from herbie.toolbox import EasyMap, pc
from herbie import paint
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

In [ ]:
hrrr = Herbie(
    "2025-09-25 12:00", 
    model="hrrr", 
    product="sfc", 
    fxx=6
)

rtma = Herbie("2025-09-25 18:00", 
           model="rtma", 
           product="anl"
)

In [26]:
ds_hrrr = hrrr.xarray("TMP:2 m above")
ds_hrrr

<xarray.Dataset> Size: 38MB
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 8B 2025-09-25T12:00:00
    step                 timedelta64[ns] 8B 06:00:00
    heightAboveGround    float64 8B 2.0
    latitude             (y, x) float64 15MB 21.14 21.15 21.15 ... 47.85 47.84
    longitude            (y, x) float64 15MB 237.3 237.3 237.3 ... 299.0 299.1
    valid_time           datetime64[ns] 8B 2025-09-25T18:00:00
    gribfile_projection  object 8B None
Dimensions without coordinates: y, x
Data variables:
    t2m                  (y, x) float32 8MB 296.0 296.0 296.0 ... 286.8 286.9
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              /Users/john_b_needham/data/hrrr/20250925/subset_...
    search:                  TMP:2 m above

In [ ]:
ds_rtma = rtma.xarray("TMP:2 m above")
ds_rtma

In [ ]:
newlat = ds_hrrr["latitude"].values
newlat

array([[21.138123  , 21.14511004, 21.1520901 , ..., 21.1545089 ,
        21.14753125, 21.14054663],
       [21.16299459, 21.1699845 , 21.17696744, ..., 21.17938723,
        21.1724067 , 21.16541921],
       [21.18786863, 21.19486142, 21.20184723, ..., 21.20426802,
        21.19728462, 21.19029425],
       ...,
       [47.78955926, 47.799849  , 47.81012868, ..., 47.81369093,
        47.80341474, 47.79312849],
       [47.81409316, 47.82438621, 47.8346692 , ..., 47.83823259,
        47.8279531 , 47.81766354],
       [47.8386235 , 47.84891986, 47.85920615, ..., 47.86277069,
        47.85248789, 47.84219502]])

In [28]:
newlon = ds_hrrr["longitude"].values
newlon

array([[237.280472  , 237.30713868, 237.3338097 , ..., 287.6569408 ,
        287.68361332, 287.71028151],
       [237.27297501, 237.29964881, 237.32632695, ..., 287.66442108,
        287.69110073, 287.71777603],
       [237.26547368, 237.2921546 , 237.31883986, ..., 287.6719057 ,
        287.69859247, 287.7252749 ],
       ...,
       [225.9351904 , 225.97171577, 226.00825329, ..., 298.97907406,
        299.0156158 , 299.05214538],
       [225.91986142, 225.95639874, 225.99294822, ..., 298.99437498,
        299.03092868, 299.06747022],
       [225.90452027, 225.94106954, 225.97763099, ..., 299.00968806,
        299.04625373, 299.08280723]])

In [31]:
rtma_on_hrrr = ds_rtma.assign_coords(
    latitude = (("y", "x"), newlat),
    longitude = (("y", "x"), newlon)
)

AlignmentError: cannot reindex or align along dimension 'y' because of conflicting dimension sizes: {1059, 1597}

In [ ]:
ax = EasyMap("50m", crs=ds_rtma.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    rtma_on_hrrr.longitude,
    rtma_on_hrrr.latitude,
    rtma_on_hrrr.t2m - 273.15,
    transform=pc,
    **paint.NWSTemperature.kwargs2,
)
plt.colorbar(
    p,
    ax=ax,
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
    **paint.NWSTemperature.cbar_kwargs2,
)

ax.set_title(
    f"{rtma_on_hrrr.model.upper()}: {rtma.product_description}\nValid: {rtma_on_hrrr.valid_time.dt.strftime('%H:%M UTC %d %b %Y').item()}",
    loc="left",
)
ax.set_title(ds_rtma.t2m.GRIB_name, loc="right")